# Phase 1: Load and Join Data

## Step 1: Load the 3 CSV files into Spark DataFrames

In [0]:
# Load Crashes dataset
crashes_df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/Final Project Data/Traffic_Crashes___Crashes_20250425.csv")

# Load Vehicles dataset
vehicles_df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/Final Project Data/Traffic_Crashes___Vehicles_20250425.csv")

# Load People dataset
people_df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/Final Project Data/Traffic_Crashes___People_20250425.csv")

## Step 2: Quick Schema and Row Count Check (small EDA)

In [0]:
# Check schemas
crashes_df.printSchema()
vehicles_df.printSchema()
people_df.printSchema()

# Check row counts
print(f"Crashes: {crashes_df.count()} rows")
print(f"Vehicles: {vehicles_df.count()} rows")
print(f"People: {people_df.count()} rows")

# Phase 2: Joins

## Step 1: Join the Datasets

Before joining, we need to rename or drop duplicate columns like CRASH_DATE from vehicles_df and people_df, because:

- crashes_df has the most accurate crash-level date

- vehicle and people versions are mostly redundant for our big-picture analysis

In [0]:
# Step 1: Drop CRASH_DATE from vehicles and people BEFORE join
vehicles_df = vehicles_df.drop("CRASH_DATE")
people_df = people_df.drop("CRASH_DATE")

# Step 2: Join Crashes and Vehicles
crash_vehicle_df = crashes_df.join(
    vehicles_df,
    on="CRASH_RECORD_ID",
    how="inner"
)

# Step 3: Join Crash-Vehicle with People
final_df = crash_vehicle_df.join(
    people_df,
    on=["CRASH_RECORD_ID", "VEHICLE_ID"],
    how="left"
)

# Step 4: Show sample
final_df.select(
    "CRASH_RECORD_ID", "CRASH_DATE", "CRASH_TYPE", 
    "VEHICLE_TYPE", "PERSON_TYPE", "INJURY_CLASSIFICATION"
).show(5, truncate=False)

## Step 2: Cache & Save the Final Joined Dataset

In [0]:
# Cache the DataFrame in memory
final_df.cache()

# Save as a Parquet file (this will go to DBFS)
final_df.write.mode("overwrite").parquet("/FileStore/Final Project Data/final_joined_crash_data.parquet")

print("✅ Final joined dataset saved as Parquet!")

# Phase 3: Primary EDA (Exploratory Data Analysis)

## Step 1: Load the Cached/Parquet Dataset

In [0]:
# Load from saved Parquet file
final_df = spark.read.parquet("/FileStore/Final Project Data/final_joined_crash_data.parquet")

# Show sample
final_df.show(5)

+--------------------+----------+----------------+--------------------+------------------+----------------------+--------------------+-----------------+--------------------+--------------------+---------------+--------+------------------+--------------------+---------------+--------------------+--------------------+----------------------+------------------+-------------+-------------+--------------------+-----------------------+----------------------+---------+----------------+-----------------+------------------+--------------+------------------+---------+-----------+--------------+-----------------+---------+--------------------+--------------+--------------+-----------------------+---------------------------+-----------------------------+----------------------+----------------+----------+-----------------+-----------+-------------+--------------+--------------------+-------------+-------+---------+--------------+----------+-------+--------------------+---------------+------------+--

## Step 2: Quick Data Overview

In [0]:
# Show schema
final_df.printSchema()

# Count number of rows and columns
row_count = final_df.count()
col_count = len(final_df.columns)

print(f"Total Rows: {row_count}")
print(f"Total Columns: {col_count}")

root
 |-- CRASH_RECORD_ID: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- CRASH_DATE_EST_I: string (nullable = true)
 |-- CRASH_DATE: string (nullable = true)
 |-- POSTED_SPEED_LIMIT: integer (nullable = true)
 |-- TRAFFIC_CONTROL_DEVICE: string (nullable = true)
 |-- DEVICE_CONDITION: string (nullable = true)
 |-- WEATHER_CONDITION: string (nullable = true)
 |-- LIGHTING_CONDITION: string (nullable = true)
 |-- FIRST_CRASH_TYPE: string (nullable = true)
 |-- TRAFFICWAY_TYPE: string (nullable = true)
 |-- LANE_CNT: integer (nullable = true)
 |-- ALIGNMENT: string (nullable = true)
 |-- ROADWAY_SURFACE_COND: string (nullable = true)
 |-- ROAD_DEFECT: string (nullable = true)
 |-- REPORT_TYPE: string (nullable = true)
 |-- CRASH_TYPE: string (nullable = true)
 |-- INTERSECTION_RELATED_I: string (nullable = true)
 |-- NOT_RIGHT_OF_WAY_I: string (nullable = true)
 |-- HIT_AND_RUN_I: string (nullable = true)
 |-- DAMAGE: string (nullable = true)
 |-- DATE_POLICE_NO

### Primary EDA Analysis Summary

**Dataset Status:**

- Total Rows: 2,308,577
- Total Columns: 143
- Source Tables Joined: Crashes + Vehicles + People
- File Format: Parquet (optimized)

## Step 3: Null Value Analysis

In [0]:
from pyspark.sql.functions import col, sum as spark_sum, when

# Calculate missing values per column
missing_counts = final_df.select([
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in final_df.columns
])

missing_counts.show(vertical=True, truncate=False)

-RECORD 0--------------------------------
 CRASH_RECORD_ID               | 0       
 VEHICLE_ID                    | 44688   
 CRASH_DATE_EST_I              | 2152465 
 CRASH_DATE                    | 0       
 POSTED_SPEED_LIMIT            | 0       
 TRAFFIC_CONTROL_DEVICE        | 0       
 DEVICE_CONDITION              | 0       
 WEATHER_CONDITION             | 0       
 LIGHTING_CONDITION            | 0       
 FIRST_CRASH_TYPE              | 0       
 TRAFFICWAY_TYPE               | 0       
 LANE_CNT                      | 1814500 
 ALIGNMENT                     | 0       
 ROADWAY_SURFACE_COND          | 0       
 ROAD_DEFECT                   | 0       
 REPORT_TYPE                   | 88042   
 CRASH_TYPE                    | 0       
 INTERSECTION_RELATED_I        | 1729878 
 NOT_RIGHT_OF_WAY_I            | 2220170 
 HIT_AND_RUN_I                 | 1605924 
 DAMAGE                        | 0       
 DATE_POLICE_NOTIFIED          | 0       
 PRIM_CONTRIBUTORY_CAUSE       | 0

### Key Observations from our Output

---

### 1. Good Columns (Almost No Missing Values)

✅ These columns are mostly complete:

- **CRASH_RECORD_ID**
- **CRASH_DATE**
- **POSTED_SPEED_LIMIT**
- **TRAFFIC_CONTROL_DEVICE**
- **DEVICE_CONDITION**
- **WEATHER_CONDITION**
- **LIGHTING_CONDITION**
- **FIRST_CRASH_TYPE**
- **CRASH_TYPE**
- **ROADWAY_SURFACE_COND**
- **PRIM_CONTRIBUTORY_CAUSE**
- **SEC_CONTRIBUTORY_CAUSE**
- **INJURIES_TOTAL**, **INJURIES_FATAL**, **INJURIES_NO_INDICATION**
- **CRASH_HOUR**, **CRASH_DAY_OF_WEEK**, **CRASH_MONTH**

➡️ **These are high-quality candidates for modeling.**

---

### 2. Columns with Massive Missing Values (>90% missing)

🚫 These columns are almost useless unless filled carefully:

- **CRASH_DATE_EST_I**
- **INTERSECTION_RELATED_I**
- **NOT_RIGHT_OF_WAY_I**
- **HIT_AND_RUN_I**
- **TOWED_I**
- **WORK_ZONE_I**, **WORK_ZONE_TYPE**
- **AREA_XX_I** columns (very sparse)
- **HAZMAT_*** columns
- **WIDE_LOAD_I**
- **EMS_RUN_NO**
- **BAC_RESULT VALUE**
- **PEDPEDAL_*** columns

➡️ **We can safely DROP many of these irrelevant or extremely sparse columns to save memory and focus modeling.**

---

### 3. Person Data Is Very Sparse

Fields like:

- **PERSON_ID**
- **AGE**
- **SEX**
- **INJURY_CLASSIFICATION**

have significant missingness (**>30% to 70%**), especially pedestrian/bicycle fields (up to **99% missing**).

➡️ **We should treat people-related data separately or optionally, and not rely heavily on them for initial modeling.**


## Step 4: Feature Shortlisting

🧠 Feature Shortlisting Strategy

We’ve categorized features into three clear groups for better modeling clarity.

---

### 🎯 Target Columns (Modeling Outcomes)

| Column               | Type                 | Reason                                                                 |
|----------------------|----------------------|------------------------------------------------------------------------|
| **INJURIES_TOTAL**   | Regression           | Total people injured — best for cost/severity prediction               |
| **INJURIES_FATAL**   | Regression / Binary  | Can be used for binary classification (e.g., fatal vs. not)           |
| **MOST_SEVERE_INJURY** | Classification     | Categorical — good for multi-class classification (e.g., NO INJURY, FATAL, etc.) |

> 💡 *We can pick any of the three depending on our modeling direction (we’ll finalize this after EDA of distributions).*

---

### ✅ Selected Input Features (Predictors)

| Feature                          | Type        | Why                                                               |
|----------------------------------|-------------|--------------------------------------------------------------------|
| **CRASH_TYPE**                   | Categorical | Type of crash: hit object, rear-end, sideswipe                    |
| **FIRST_CRASH_TYPE**             | Categorical | More detailed breakdown                                           |
| **POSTED_SPEED_LIMIT**           | Numeric     | Environmental risk factor                                         |
| **TRAFFIC_CONTROL_DEVICE**       | Categorical | Controls in place                                                 |
| **DEVICE_CONDITION**             | Categorical | Broken/non-functional signals?                                    |
| **WEATHER_CONDITION**            | Categorical | Snow, rain, fog, etc.                                             |
| **LIGHTING_CONDITION**           | Categorical | Daylight vs. dark                                                 |
| **ROADWAY_SURFACE_COND**         | Categorical | Dry, wet, snowy, icy                                              |
| **PRIM_CONTRIBUTORY_CAUSE**      | Categorical | Top reason of crash                                               |
| **SEC_CONTRIBUTORY_CAUSE**       | Categorical | Secondary factor                                                  |
| **CRASH_HOUR**, **DAY_OF_WEEK**, **MONTH** | Numeric | Temporal patterns                                           |
| **VEHICLE_TYPE**                 | Categorical | Vehicle class involved                                            |
| **MANEUVER**                     | Categorical | Turning, going straight, etc.                                     |

> ✅ *These columns are mostly complete and provide useful signal for predicting injury or fatality risk.*

---

### 🗑️ To Drop (Too Sparse or Irrelevant)

| Reason                | Examples                                                                 |
|------------------------|--------------------------------------------------------------------------|
| 🚫 **90%+ Missing**    | INTERSECTION_RELATED_I, HIT_AND_RUN_I, TOWED_I, AREA_XX_I, HAZMAT_       |
| 📉 **Sparse person fields** | EMS_AGENCY, PEDPEDAL_ACTION, BAC_RESULT VALUE, DRIVERS_LICENSE_CLASS  |
| 🔁 **Redundant**       | STREET_NO, STREET_DIRECTION, STREET_NAME, LOCATION (too specific/geolocation not needed) |


In [0]:
# Create Cleaned Feature Dataset

# Choose selected columns
selected_columns = [
    "INJURIES_TOTAL", "INJURIES_FATAL", "MOST_SEVERE_INJURY",
    "CRASH_TYPE", "FIRST_CRASH_TYPE", "POSTED_SPEED_LIMIT",
    "TRAFFIC_CONTROL_DEVICE", "DEVICE_CONDITION", "WEATHER_CONDITION",
    "LIGHTING_CONDITION", "ROADWAY_SURFACE_COND", "PRIM_CONTRIBUTORY_CAUSE",
    "SEC_CONTRIBUTORY_CAUSE", "CRASH_HOUR", "CRASH_DAY_OF_WEEK", "CRASH_MONTH",
    "VEHICLE_TYPE", "MANEUVER"
]

# Filter the full DataFrame to only include selected columns
model_df = final_df.select(*selected_columns)

# Preview it
model_df.show(5)

+--------------+--------------+--------------------+--------------------+--------------------+------------------+----------------------+--------------------+-----------------+--------------------+--------------------+-----------------------+----------------------+----------+-----------------+-----------+--------------------+--------------+
|INJURIES_TOTAL|INJURIES_FATAL|  MOST_SEVERE_INJURY|          CRASH_TYPE|    FIRST_CRASH_TYPE|POSTED_SPEED_LIMIT|TRAFFIC_CONTROL_DEVICE|    DEVICE_CONDITION|WEATHER_CONDITION|  LIGHTING_CONDITION|ROADWAY_SURFACE_COND|PRIM_CONTRIBUTORY_CAUSE|SEC_CONTRIBUTORY_CAUSE|CRASH_HOUR|CRASH_DAY_OF_WEEK|CRASH_MONTH|        VEHICLE_TYPE|      MANEUVER|
+--------------+--------------+--------------------+--------------------+--------------------+------------------+----------------------+--------------------+-----------------+--------------------+--------------------+-----------------------+----------------------+----------+-----------------+-----------+-----------

# Phase 4: Final EDA & Modeling Direction

## Step 1: Quick Class Distribution of MOST_SEVERE_INJURY

In [0]:
model_df.groupBy("MOST_SEVERE_INJURY").count().orderBy("count", ascending=False).show(truncate=False)

+------------------------+-------+
|MOST_SEVERE_INJURY      |count  |
+------------------------+-------+
|NO INDICATION OF INJURY |1917413|
|NONINCAPACITATING INJURY|214930 |
|REPORTED, NOT EVIDENT   |124303 |
|INCAPACITATING INJURY   |45339  |
|NULL                    |3439   |
|FATAL                   |3153   |
+------------------------+-------+



### Interpretation of the results

---

### Step 1: MOST_SEVERE_INJURY Class Distribution

| Class                     | Count     | % of Total  |
|----------------------------|-----------|-------------|
| **NO INDICATION OF INJURY** | 1,917,413 | ~83%        |
| **NONINCAPACITATING INJURY** | 214,930  | ~9%         |
| **REPORTED, NOT EVIDENT**   | 124,303   | ~5%         |
| **INCAPACITATING INJURY**   | 45,339    | ~2%         |
| **FATAL**                   | 3,153     | ~0.13%      |
| **NULL (missing values)**   | 3,439     | ~0.15%      |

---

### ✅ Key Observations:

- Strong class imbalance: Most records (~83%) are **"No Injury."**
- Fatal injuries are extremely rare (~0.13%).
- Missing labels (NULL) are small (~0.15%) → **We can drop these rows safely.**


## Step 2: Distribution of INJURIES_TOTAL

In [0]:
# Descriptive stats
model_df.select("INJURIES_TOTAL").describe().show()

# Quantile breakdown
quantiles = model_df.approxQuantile("INJURIES_TOTAL", [0.5, 0.9, 0.99], 0.01)
print(f"Median: {quantiles[0]}, 90th percentile: {quantiles[1]}, 99th percentile: {quantiles[2]}")

+-------+------------------+
|summary|    INJURIES_TOTAL|
+-------+------------------+
|  count|           2305165|
|   mean|0.2680302711519566|
| stddev|0.7659568492303127|
|    min|                 0|
|    max|                21|
+-------+------------------+

Median: 0.0, 90th percentile: 1.0, 99th percentile: 21.0


### Interpretation of the results

---

### INJURIES_TOTAL Distribution

| Metric             | Value  |
|--------------------|--------|
| **Mean**           | ~0.27  |
| **Median**         | 0.0    |
| **90th Percentile** | 1.0    |
| **99th Percentile** | 21.0   |

---

### ✅ Key Observations:

- **Highly skewed:** Most crashes have zero injuries.
- **Rare but extreme cases:** Some crashes involve up to 21 people injured.
- **Modeling impact:** If you model **INJURIES_TOTAL**, it will be highly imbalanced, and simple linear regression might perform poorly without transformation or special techniques.

## Step 3: Save Trimmed Modeling Dataset

In [0]:
model_df.write.mode("overwrite").parquet("/FileStore/Final Project Data/final_model_df.parquet")
print("✅ Saved trimmed modeling dataset as Parquet.")

✅ Saved trimmed modeling dataset as Parquet.


## Step 4: Modeling Direction Summary

### 🎯 Research Question

Can we predict the most severe injury outcome of a traffic crash based on crash conditions, vehicle types, roadway environment, and crash time?

---

### 📋 Modeling Pathways

| Direction                         | Target Variable       | Method                                              | Rationale                                                                 |
|-----------------------------------|------------------------|-----------------------------------------------------|--------------------------------------------------------------------------|
| ✅ **Multi-class Classification (Preferred)** | MOST_SEVERE_INJURY    | Logistic Regression, Decision Tree, Random Forest, GBTClassifier | Classes are well-defined; manageable imbalance after slight adjustments; strong interpretability |
| 🔵 **Binary Classification (Secondary Option)** | INJURIES_FATAL (Fatal vs Non-Fatal) | Logistic Regression, Tree models | Predicting fatal crashes is important but requires rebalancing techniques |
| 🔴 **Regression (Not Preferred)** | INJURIES_TOTAL         | Linear Regression, GBT Regression                   | Highly skewed, would require heavy transformation (not ideal for current project scope) |

---

### 📌 Final Recommendation

- Proceed with **Multi-Class Classification** on **MOST_SEVERE_INJURY**.
- Drop rows where **MOST_SEVERE_INJURY** is **NULL**.
- (Optional) Combine rare classes (e.g., **FATAL** + **INCAPACITATING** → **SEVERE_INJURY**) if we want a simpler 3-class setup:
  - **No Injury**
  - **Minor Injury**
  - **Severe Injury**

---

### 📋 Final Selected Models

| Model                     | Reason                                                         |
|----------------------------|----------------------------------------------------------------|
| **Logistic Regression**    | Baseline performance check                                     |
| **Decision Tree Classifier** | Understand simple rules that cause injuries                   |
| **Random Forest Classifier** | Boost accuracy and feature understanding                      |
| **GBTClassifier**          | High-performance ensemble method                              |

---

### ✅ Quick Final Code Adjustment for Team

Before starting modeling, **drop NULLs** in the target:

```python
# Drop rows with missing MOST_SEVERE_INJURY
model_df_clean = model_df.filter(model_df["MOST_SEVERE_INJURY"].isNotNull())
